# FaceWatch 프로젝트 분석 노트북

이 노트북은 FaceWatch 프로젝트의 임베딩 데이터를 확인하고 분석하는 도구입니다.

## 주요 기능
- 임베딩 파일 구조 확인
- 갤러리 로드 및 통계 확인
- Bank vs Centroid 비교
- 인물 간 유사도 분석
- 임베딩 분포 시각화


In [ ]:
# 프로젝트 루트 디렉토리로 작업 디렉토리 설정
import os
import sys
from pathlib import Path

# notebooks 폴더에서 실행 중이면 상위 폴더로 이동
if Path.cwd().name == "notebooks":
    os.chdir("..")
    print(f"작업 디렉토리: {os.getcwd()}")
else:
    print(f"현재 작업 디렉토리: {os.getcwd()}")


In [ ]:
# 필요한 라이브러리 import
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict
import pandas as pd

# 프로젝트 유틸리티 import
sys.path.insert(0, str(Path("src")))
from utils.gallery_loader import load_gallery, match_with_bank, match_with_bank_detailed, l2_normalize

# 시각화 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10


## 1. 임베딩 파일 구조 확인


In [ ]:
# 임베딩 폴더 확인
emb_dir = Path("outputs/embeddings")

if not emb_dir.exists():
    print(f"[ERROR] 임베딩 폴더를 찾을 수 없음: {emb_dir}")
    print("먼저 face_enroll_final.py를 실행하여 임베딩을 생성하세요.")
else:
    print(f"[OK] 임베딩 폴더: {emb_dir}")
    print()
    
    # 사람별 폴더 확인
    person_dirs = [d for d in emb_dir.iterdir() if d.is_dir()]
    print(f"등록된 인물 수: {len(person_dirs)}")
    print()
    
    for person_dir in sorted(person_dirs):
        person_id = person_dir.name
        bank_path = person_dir / "bank.npy"
        centroid_path = person_dir / "centroid.npy"
        
        print(f"[{person_id}]")
        if bank_path.exists():
            bank = np.load(bank_path)
            print(f"   - bank.npy: {bank.shape} ({bank.shape[0]}개 임베딩)")
        if centroid_path.exists():
            centroid = np.load(centroid_path)
            print(f"   - centroid.npy: {centroid.shape}")
        print()


## 2. 갤러리 로드 및 통계 확인


In [ ]:
# 갤러리 로드 (Bank 우선)
gallery = load_gallery(emb_dir, use_bank=True)

print(f"갤러리 로드 완료: {len(gallery)}명")
print()

# 각 인물의 임베딩 정보 출력
gallery_stats = []
for person_id, emb_data in sorted(gallery.items()):
    if emb_data.ndim == 2:
        emb_type = "Bank"
        emb_count = emb_data.shape[0]
        emb_dim = emb_data.shape[1]
    else:
        emb_type = "Centroid"
        emb_count = 1
        emb_dim = emb_data.shape[0]
    
    gallery_stats.append({
        "person_id": person_id,
        "type": emb_type,
        "count": emb_count,
        "dimension": emb_dim
    })
    
    print(f"{person_id:10s}: {emb_type:8s} ({emb_count:3d}개 임베딩, {emb_dim}차원)")

# 통계를 DataFrame으로 변환
df_stats = pd.DataFrame(gallery_stats)
print()
print("=" * 60)
print("갤러리 통계 요약")
print("=" * 60)
print(df_stats.to_string(index=False))


## 3. Bank vs Centroid 비교


In [ ]:
# 특정 인물의 Bank와 Centroid 비교
person_id = "hani"  # 비교할 인물 ID 변경 가능

person_dir = emb_dir / person_id
bank_path = person_dir / "bank.npy"
centroid_path = person_dir / "centroid.npy"

if bank_path.exists() and centroid_path.exists():
    bank = np.load(bank_path)
    centroid = np.load(centroid_path)
    
    # Centroid 계산 (Bank의 평균)
    calculated_centroid = bank.mean(axis=0)
    calculated_centroid = l2_normalize(calculated_centroid)
    
    print(f"인물: {person_id}")
    print(f"Bank 크기: {bank.shape}")
    print(f"Centroid 크기: {centroid.shape}")
    print()
    
    # 저장된 Centroid와 계산된 Centroid 비교
    similarity = float(np.dot(centroid, calculated_centroid))
    print(f"저장된 Centroid vs 계산된 Centroid 유사도: {similarity:.6f}")
    print()
    
    # Bank 내부 임베딩 간 유사도 분포
    if bank.shape[0] > 1:
        bank_normalized = bank / (np.linalg.norm(bank, axis=1, keepdims=True) + 1e-6)
        similarities = np.dot(bank_normalized, bank_normalized.T)
        # 대각선 제거 (자기 자신과의 유사도)
        mask = ~np.eye(similarities.shape[0], dtype=bool)
        bank_similarities = similarities[mask]
        
        print(f"Bank 내부 임베딩 간 유사도 통계:")
        print(f"  평균: {bank_similarities.mean():.4f}")
        print(f"  최소: {bank_similarities.min():.4f}")
        print(f"  최대: {bank_similarities.max():.4f}")
        print(f"  표준편차: {bank_similarities.std():.4f}")
else:
    print(f"[ERROR] {person_id}의 Bank 또는 Centroid 파일을 찾을 수 없습니다.")


## 4. 인물 간 유사도 매트릭스


In [ ]:
# 모든 인물 간 유사도 계산
person_ids = sorted(gallery.keys())
n_persons = len(person_ids)

if n_persons > 1:
    similarity_matrix = np.zeros((n_persons, n_persons))
    
    for i, person_id1 in enumerate(person_ids):
        for j, person_id2 in enumerate(person_ids):
            if i == j:
                similarity_matrix[i, j] = 1.0
            else:
                emb1 = gallery[person_id1]
                emb2 = gallery[person_id2]
                
                # Bank인 경우 최대 유사도 사용
                if emb1.ndim == 2:
                    sims1 = np.dot(emb1, emb2.T if emb2.ndim == 2 else emb2)
                    max_sim = float(np.max(sims1))
                elif emb2.ndim == 2:
                    sims2 = np.dot(emb1, emb2.T)
                    max_sim = float(np.max(sims2))
                else:
                    max_sim = float(np.dot(emb1, emb2))
                
                similarity_matrix[i, j] = max_sim
    
    # 히트맵 시각화
    plt.figure(figsize=(10, 8))
    sns.heatmap(similarity_matrix, 
                xticklabels=person_ids, 
                yticklabels=person_ids,
                annot=True, 
                fmt='.3f',
                cmap='RdYlBu_r',
                vmin=0, 
                vmax=1,
                square=True,
                cbar_kws={'label': 'Cosine Similarity'})
    plt.title('인물 간 유사도 매트릭스', fontsize=14, fontweight='bold')
    plt.xlabel('인물 ID', fontsize=12)
    plt.ylabel('인물 ID', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    # 통계 출력
    print("인물 간 유사도 통계:")
    mask = ~np.eye(n_persons, dtype=bool)
    off_diagonal = similarity_matrix[mask]
    print(f"  평균: {off_diagonal.mean():.4f}")
    print(f"  최소: {off_diagonal.min():.4f}")
    print(f"  최대: {off_diagonal.max():.4f}")
    print(f"  표준편차: {off_diagonal.std():.4f}")
else:
    print("인물이 1명만 있어서 유사도 비교를 할 수 없습니다.")


## 5. 임베딩 값 분포 시각화


In [ ]:
# 특정 인물의 임베딩 분포 확인
person_id = "hani"  # 확인할 인물 ID 변경 가능

if person_id in gallery:
    emb_data = gallery[person_id]
    
    # Centroid인 경우
    if emb_data.ndim == 1:
        emb = emb_data
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.hist(emb, bins=50, color="steelblue", alpha=0.7, edgecolor='black')
        plt.title(f"{person_id} - Centroid 임베딩 값 분포", fontweight='bold')
        plt.xlabel("값")
        plt.ylabel("빈도")
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.plot(emb, alpha=0.7, linewidth=0.5)
        plt.title(f"{person_id} - Centroid 임베딩 벡터", fontweight='bold')
        plt.xlabel("차원 인덱스")
        plt.ylabel("값")
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f"통계:")
        print(f"  평균: {emb.mean():.6f}")
        print(f"  표준편차: {emb.std():.6f}")
        print(f"  최소값: {emb.min():.6f}")
        print(f"  최대값: {emb.max():.6f}")
        print(f"  L2 norm: {np.linalg.norm(emb):.6f}")
    
    # Bank인 경우
    else:
        print(f"{person_id} - Bank 임베딩 ({emb_data.shape[0]}개)")
        
        # 각 임베딩의 통계
        means = emb_data.mean(axis=1)
        stds = emb_data.std(axis=1)
        norms = np.linalg.norm(emb_data, axis=1)
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # 히스토그램: 모든 임베딩 값
        axes[0, 0].hist(emb_data.flatten(), bins=50, color="steelblue", alpha=0.7, edgecolor='black')
        axes[0, 0].set_title("모든 임베딩 값 분포", fontweight='bold')
        axes[0, 0].set_xlabel("값")
        axes[0, 0].set_ylabel("빈도")
        axes[0, 0].grid(True, alpha=0.3)
        
        # 각 임베딩의 평균값
        axes[0, 1].bar(range(len(means)), means, color="coral", alpha=0.7, edgecolor='black')
        axes[0, 1].set_title("각 임베딩의 평균값", fontweight='bold')
        axes[0, 1].set_xlabel("임베딩 인덱스")
        axes[0, 1].set_ylabel("평균값")
        axes[0, 1].grid(True, alpha=0.3)
        
        # 각 임베딩의 표준편차
        axes[1, 0].bar(range(len(stds)), stds, color="mediumseagreen", alpha=0.7, edgecolor='black')
        axes[1, 0].set_title("각 임베딩의 표준편차", fontweight='bold')
        axes[1, 0].set_xlabel("임베딩 인덱스")
        axes[1, 0].set_ylabel("표준편차")
        axes[1, 0].grid(True, alpha=0.3)
        
        # 각 임베딩의 L2 norm
        axes[1, 1].bar(range(len(norms)), norms, color="gold", alpha=0.7, edgecolor='black')
        axes[1, 1].set_title("각 임베딩의 L2 Norm", fontweight='bold')
        axes[1, 1].set_xlabel("임베딩 인덱스")
        axes[1, 1].set_ylabel("L2 Norm")
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n통계:")
        print(f"  전체 평균: {emb_data.mean():.6f}")
        print(f"  전체 표준편차: {emb_data.std():.6f}")
        print(f"  L2 norm 평균: {norms.mean():.6f}")
else:
    print(f"[ERROR] {person_id}를 갤러리에서 찾을 수 없습니다.")


## 6. 테스트 얼굴과 갤러리 매칭 테스트


In [ ]:
# InsightFace를 사용하여 테스트 이미지에서 얼굴 임베딩 추출
from insightface.app import FaceAnalysis
from utils.device_config import get_device_id, safe_prepare_insightface
import cv2

# 테스트 이미지 경로 (변경 가능)
test_image_path = "images/source/test.jpg"  # 또는 다른 이미지 경로

if Path(test_image_path).exists():
    # InsightFace 초기화
    device_id = get_device_id()
    app = FaceAnalysis(name="buffalo_l")
    safe_prepare_insightface(app, device_id, det_size=(640, 640))
    
    # 이미지 로드
    img = cv2.imread(test_image_path)
    if img is None:
        print(f"[ERROR] 이미지를 읽을 수 없음: {test_image_path}")
    else:
        print(f"[OK] 이미지 로드: {test_image_path}")
        print(f"   이미지 크기: {img.shape}")
        print()
        
        # 얼굴 검출
        faces = app.get(img)
        print(f"감지된 얼굴 수: {len(faces)}")
        print()
        
        if len(faces) > 0:
            # 각 얼굴에 대해 매칭 테스트
            for i, face in enumerate(faces):
                face_emb = face.embedding.astype("float32")
                
                # 상세 매칭 정보
                best_id, best_sim, second_sim = match_with_bank_detailed(face_emb, gallery)
                sim_gap = best_sim - second_sim if second_sim > -1 else best_sim
                
                print(f"[얼굴 {i+1}]")
                print(f"  매칭 결과: {best_id}")
                print(f"  최고 유사도: {best_sim:.4f}")
                print(f"  두 번째 유사도: {second_sim:.4f}")
                print(f"  유사도 차이: {sim_gap:.4f}")
                print()
                
                # 임계값과 비교
                BASE_THRESH = 0.30
                is_match = best_sim >= BASE_THRESH and sim_gap >= 0.05
                match_status = "[MATCH]" if is_match else "[NO MATCH]"
                print(f"  임계값 ({BASE_THRESH}) 이상: {match_status}")
                print()
        else:
            print("[WARNING] 얼굴을 감지하지 못했습니다.")
else:
    print(f"[ERROR] 테스트 이미지를 찾을 수 없음: {test_image_path}")
    print("다른 이미지 경로를 지정하거나 이미지를 준비하세요.")


## 7. 분석 결과 요약


In [ ]:
# 전체 분석 결과 요약
print("=" * 70)
print("FaceWatch 프로젝트 분석 결과 요약")
print("=" * 70)
print()

print(f"갤러리 정보:")
print(f"   등록된 인물 수: {len(gallery)}명")
print()

# Bank vs Centroid 통계
bank_count = sum(1 for emb in gallery.values() if emb.ndim == 2)
centroid_count = len(gallery) - bank_count
print(f"   Bank 사용: {bank_count}명")
print(f"   Centroid 사용: {centroid_count}명")
print()

# 임베딩 개수 통계
total_embeddings = sum(emb.shape[0] if emb.ndim == 2 else 1 for emb in gallery.values())
print(f"   총 임베딩 수: {total_embeddings}개")
print()

# 인물 간 유사도 통계 (이미 계산된 경우)
if len(gallery) > 1:
    person_ids = sorted(gallery.keys())
    similarity_matrix = np.zeros((len(person_ids), len(person_ids)))
    
    for i, person_id1 in enumerate(person_ids):
        for j, person_id2 in enumerate(person_ids):
            if i == j:
                similarity_matrix[i, j] = 1.0
            else:
                emb1 = gallery[person_id1]
                emb2 = gallery[person_id2]
                
                if emb1.ndim == 2:
                    sims1 = np.dot(emb1, emb2.T if emb2.ndim == 2 else emb2)
                    max_sim = float(np.max(sims1))
                elif emb2.ndim == 2:
                    sims2 = np.dot(emb1, emb2.T)
                    max_sim = float(np.max(sims2))
                else:
                    max_sim = float(np.dot(emb1, emb2))
                
                similarity_matrix[i, j] = max_sim
    
    mask = ~np.eye(len(person_ids), dtype=bool)
    off_diagonal = similarity_matrix[mask]
    
    print(f"인물 간 유사도:")
    print(f"   평균: {off_diagonal.mean():.4f}")
    print(f"   최소: {off_diagonal.min():.4f}")
    print(f"   최대: {off_diagonal.max():.4f}")
    print()

print("=" * 70)
